In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
DATA_DIR = 'datasets/ml-data/'

In [ ]:
df = pd.read_csv(DATA_DIR + 'ratings.csv')

In [ ]:
df.head()

In [ ]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .config('spark.driver.memory', '8G')
    .config('spark.sql.analyzer.failAmbiguousSelfJoin', 'False')
    .master("local[*]")
    .getOrCreate()
)

In [ ]:
# считываем данные из CSV
# и преобразуем времяпроставления оценки из целого числа в дату мл временем
import os
import pyspark.sql.functions as sql_func

ratings = (
    spark
    .read
    .csv(
        os.path.join(DATA_DIR, 'ratings.csv'),
        header=True,
        inferSchema=True
    )
    # если используется меньше памяти,
    # то здесь можно взять не все данные, а небольшую выборку
    # даже при fraction=.01 качественная картина не меняеся
    .sample(withReplacement=False, fraction=1.0, seed=0)
    .withColumn('rating_datetime', sql_func.from_unixtime('timestamp'))
    .drop('timestamp')
    .cache()
)

In [ ]:
ratings.show()

In [ ]:
# оцениваем размеры данных
print('всего пользователей', ratings.select('userId').distinct().count())
print('всего фильмов', ratings.select('movieId').distinct().count())
print('всего оценок', ratings.count())

In [ ]:
# достаточно хорошим baseline является пердсказать среднюю оценку
mean_rating = ratings.agg(sql_func.avg('rating')).first()[0]
print('Средняя оценка:', mean_rating)

In [ ]:
# Функция, с помощью которой мы будем вычислять RMSE на обучающей выборке
from pyspark.sql import DataFrame
import numpy as np

def simple_evaluate(predictions_df: DataFrame) -> float:
    return np.sqrt(
        ratings
        .join(
            predictions_df,
            ['movieId', 'userId']
        ).select(
            sql_func.pow(
                ratings.rating - predictions_df.prediction,
                2
            ).alias('squared_error')
        )
        .agg(sql_func.avg('squared_error'))
        .first()[0]
    )

In [ ]:
# рекомендуем любому пользователю любой фильм случайно
mean_predictions = ratings.withColumn('prediction', sql_func.lit(mean_rating))
print('Ошибка предсказания:', simple_evaluate(mean_predictions))

In [ ]:
# посмотрим на распределение срудних оценок фильмов
movie_ratings = (
    ratings
    .groupBy('movieId')
    .agg(sql_func.avg('rating').alias('avg_movie_rating'))
    .cache()
)

In [ ]:
# фильм бывает более или менее популярным - выбросы есть в обе стороны,
# но "средних" фильмов все-таки большинство
histogram = (
    movie_ratings
    .select((.5 * sql_func.ceil(2 * sql_func.col('avg_movie_rating')))
            .alias('avg_movie_rating'))
    .groupBy('avg_movie_rating')
    .agg(sql_func.count('avg_movie_rating').alias('cnt'))
    .orderBy('avg_movie_rating')
    .toPandas()
)
sns.set()
histogram.plot(x='avg_movie_rating', y='cnt')
plt.show()

In [ ]:
# Рекомендуем наиболее популярные фильмы
avg_movie_rating_predictions = (
    ratings
    .join(movie_ratings, 'movieId')
    .select(
        ratings.movieId,
        ratings.userId,
        ratings.rating,
        movie_ratings.avg_movie_rating.alias('prediction')
    )
)
print('ошибка предсказания', simple_evaluate(avg_movie_rating_predictions))

In [ ]:
# у разных пользователей разные распределение оценок
# кто-то более придирчив, а кто-то всем ставить пятерки
user_ratings = (
    ratings
    .groupBy('userId')
    .agg(sql_func.avg('rating').alias('avg_user_rating'))
    .cache()
)

In [ ]:
# фильм бывает более или менее популярным - выбросы есть в обе стороны,
# но "средних" фильмов все-таки большинство
histogram = (
    user_ratings
    .select((.5 * sql_func.ceil(2 * sql_func.col('avg_user_rating')))
            .alias('avg_user_rating'))
    .groupBy('avg_user_rating')
    .agg(sql_func.count('avg_user_rating').alias('cnt'))
    .orderBy('avg_user_rating')
    .toPandas()
)
sns.set()
histogram.plot(x='avg_user_rating', y='cnt')
plt.show()

In [ ]:
# а может не полусумму, а подберем коэффициенты 
# с помощью линейной регрессии
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

train = (
    VectorAssembler(
        inputCols = ['avg_movie_rating', 'avg_user_rating'],
        outputCol = 'features'
    ).transform(
        ratings
        .join(movie_ratings, 'movieId')
        .join(user_ratings, 'userId')
    )
    .withColumnRenamed('rating', 'label')
    .select('movieId', 'userId', 'label', 'features')
    .cache()
)
linear_model = LinearRegression().fit(train)
stacked_prediction = (
    ratings
    .join(linear_model.transform(train), ['movieId', 'userId'])
    .select('movieId', 'userId', 'prediction')
)
print('ошибка предсказания:', simple_evaluate(stacked_prediction))

In [ ]:
# получаем формулу для предсказания оценки, которую можно использовать для рекомендаций
print(
    '[на сколько пользователь оценит фильм] = {} + {} * [средняя оценка фильма] + {} * [средняя оценка из поставленных этим пользователем]'
    .format(
        round(linear_model.intercept, 2),
        round(linear_model.coefficients[0], 2),
        round(linear_model.coefficients[1], 2)
    )
    )    